In [0]:
# 1.Create a new notebook.
# load both the csv and json files. union them properly
# Instead of dropping the duplicates, create a boolean column is_duplicate and set False to only one and True to the rest of them

# 2.Calculate mean salary and check if it is greater or equal to the salary of employees in each department.
# Calculate mean salary and check if it is greater or equal to the salary of all employees.

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
%fs ls dbfs:/FileStore/tables/

path,name,size,modificationTime
dbfs:/FileStore/tables/Drugs_package-1.csv,Drugs_package-1.csv,27234691,1707889333000
dbfs:/FileStore/tables/Drugs_package.csv,Drugs_package.csv,27234691,1707889300000
dbfs:/FileStore/tables/Drugs_product-1.csv,Drugs_product-1.csv,38743242,1707889337000
dbfs:/FileStore/tables/Drugs_product.csv,Drugs_product.csv,38743242,1707889319000
dbfs:/FileStore/tables/csv/,csv/,0,0
dbfs:/FileStore/tables/json/,json/,0,0
dbfs:/FileStore/tables/parquet/,parquet/,0,0
dbfs:/FileStore/tables/train/,train/,0,0


In [0]:
df_csv = spark.read.format("csv").option("header",True).load("dbfs:/FileStore/tables/csv/batch.csv")
df_csv.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- department: string (nullable = true)



In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType, StringType,DateType
sch = StructType([
    StructField("id",IntegerType()),
    StructField("name",StringType()),
    StructField("dob",DateType()),
    StructField("age",IntegerType()),
    StructField("salary",IntegerType()),
    StructField("department",StringType()),
])


In [0]:
df_csv = spark.read.format("csv").schema(sch).option("header",True).load("dbfs:/FileStore/tables/csv/batch.csv")
df_csv.show()

+---+-----+----------+----+------+----------+
| id| name|       dob| age|salary|department|
+---+-----+----------+----+------+----------+
|  1| John|1992-05-12|  30| 70000|        IT|
|  2|Alice|1997-02-28|  25| 60000|        HR|
|  3|  Bob|      null|null| 80000|        IT|
|  4|Emily|1994-11-22|  28| 65000|   Finance|
+---+-----+----------+----+------+----------+



In [0]:

df_json = spark.read.format("json").schema(sch).option("header",True).load("dbfs:/FileStore/tables/json")
df_json.show()

+---+------+----------+----+------+----------+
| id|  name|       dob| age|salary|department|
+---+------+----------+----+------+----------+
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  5| David|1981-12-18|  41| 90000|        HR|
|  6| Susan|1989-07-05|  33| 75000|   Finance|
|  7|  Mike|1976-03-15|  46| 95000|        IT|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  5| David|1981-12-18|  41| 90000|        HR|
|  6| Susan|1989-07-05|  33| 75000|   Finance|
|  7|  Mike|1976-03-15|  46| 95000|        IT|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|
|  2| Alice|1997-02-28|  25| 90000|   Finance|
|  2| Alice|1997-02-28|  25| 90000|   Finance|
|  4| Emily|1

In [0]:
df=df_csv.union(df_json)
df.show()

+---+------+----------+----+------+----------+
| id|  name|       dob| age|salary|department|
+---+------+----------+----+------+----------+
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  5| David|1981-12-18|  41| 90000|        HR|
|  6| Susan|1989-07-05|  33| 75000|   Finance|
|  7|  Mike|1976-03-15|  46| 95000|        IT|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  5| David|1981-12-18|  41| 90000|        HR|
|  6| Susan|1989-07-05|  33| 75000|   Finance|
|  7|  Mike|1976-03-15|  46| 95000|        IT|
| 10|Sophie|1

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, count, avg
 
windowSpec = Window.partitionBy([col(x) for x in df.columns])
df = df.withColumn("count", count("*").over(windowSpec))
df = df.withColumn("isDuplicate", col("count") > 1)
df = df.drop("count")
df.show()
     

+---+-----+----------+----+------+----------+-----------+
| id| name|       dob| age|salary|department|isDuplicate|
+---+-----+----------+----+------+----------+-----------+
|  1| John|1992-05-12|  30| 70000|        IT|       true|
|  1| John|1992-05-12|  30| 70000|        IT|       true|
|  1| John|1992-05-12|  30| 70000|        IT|       true|
|  1| John|1992-05-12|  30| 70000|        IT|       true|
|  1| John|1992-05-12|  30| 70000|        IT|       true|
|  2|Alice|1997-02-28|  25| 60000|        HR|       true|
|  2|Alice|1997-02-28|  25| 60000|        HR|       true|
|  2|Alice|1997-02-28|  25| 60000|        HR|       true|
|  2|Alice|1997-02-28|  25| 90000|   Finance|       true|
|  2|Alice|1997-02-28|  25| 90000|   Finance|       true|
|  3|  Bob|      null|null| 80000|        IT|       true|
|  3|  Bob|      null|null| 80000|        IT|       true|
|  3|  Bob|      null|null| 80000|        IT|       true|
|  4|Emily|1994-11-22|  28| 65000|   Finance|       true|
|  4|Emily|199

In [0]:
df = df.orderBy("id")
df.show()

+---+-----+----------+----+------+----------+-----------+
| id| name|       dob| age|salary|department|isDuplicate|
+---+-----+----------+----+------+----------+-----------+
|  1| John|1992-05-12|  30| 70000|        IT|       true|
|  1| John|1992-05-12|  30| 70000|        IT|       true|
|  1| John|1992-05-12|  30| 70000|        IT|       true|
|  1| John|1992-05-12|  30| 70000|        IT|       true|
|  1| John|1992-05-12|  30| 70000|        IT|       true|
|  2|Alice|1997-02-28|  25| 60000|        HR|       true|
|  2|Alice|1997-02-28|  25| 60000|        HR|       true|
|  2|Alice|1997-02-28|  25| 60000|        HR|       true|
|  2|Alice|1997-02-28|  25| 90000|   Finance|       true|
|  2|Alice|1997-02-28|  25| 90000|   Finance|       true|
|  3|  Bob|      null|null| 80000|        IT|       true|
|  3|  Bob|      null|null| 80000|        IT|       true|
|  3|  Bob|      null|null| 80000|        IT|       true|
|  4|Emily|1994-11-22|  28| 65000|   Finance|       true|
|  4|Emily|199

In [0]:
df = df.dropDuplicates()
df.show()


+---+------+----------+----+------+----------+-----------+
| id|  name|       dob| age|salary|department|isDuplicate|
+---+------+----------+----+------+----------+-----------+
|  1|  John|1992-05-12|  30| 70000|        IT|       true|
|  2| Alice|1997-02-28|  25| 60000|        HR|       true|
|  2| Alice|1997-02-28|  25| 90000|   Finance|       true|
|  3|   Bob|      null|null| 80000|        IT|       true|
|  4| Emily|1994-11-22|  28| 65000|   Finance|       true|
|  4| Emily|1994-11-22|  28| 70000|   Finance|       true|
|  5| David|1981-12-18|  41| 90000|        HR|       true|
|  6| Susan|1989-07-05|  33| 75000|   Finance|       true|
|  7|  Mike|1976-03-15|  46| 95000|        IT|       true|
|  8|  Lisa|1995-08-20|  27| 58000|        HR|       true|
|  9| James|1983-10-14|  39| 87000|        IT|       true|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|       true|
+---+------+----------+----+------+----------+-----------+



In [0]:
windowSpec = Window.partitionBy(df.department)
df = df.withColumn("Mean salary", avg("salary").over(windowSpec))
df.show()

+---+------+----------+----+------+----------+-----------+-----------------+
| id|  name|       dob| age|salary|department|isDuplicate|      Mean salary|
+---+------+----------+----+------+----------+-----------+-----------------+
|  2| Alice|1997-02-28|  25| 90000|   Finance|       true|          72400.0|
|  4| Emily|1994-11-22|  28| 65000|   Finance|       true|          72400.0|
|  4| Emily|1994-11-22|  28| 70000|   Finance|       true|          72400.0|
|  6| Susan|1989-07-05|  33| 75000|   Finance|       true|          72400.0|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|       true|          72400.0|
|  2| Alice|1997-02-28|  25| 60000|        HR|       true|69333.33333333333|
|  5| David|1981-12-18|  41| 90000|        HR|       true|69333.33333333333|
|  8|  Lisa|1995-08-20|  27| 58000|        HR|       true|69333.33333333333|
|  1|  John|1992-05-12|  30| 70000|        IT|       true|          83000.0|
|  3|   Bob|      null|null| 80000|        IT|       true|          83000.0|

In [0]:
df = df.withColumn(
    "Is Salary greater than mean Salary",
    col("Mean salary") < col("salary")
).orderBy("id")
df.show()

+---+------+----------+----+------+----------+-----------+-----------------+----------------------------------+
| id|  name|       dob| age|salary|department|isDuplicate|      Mean salary|Is Salary greater than mean Salary|
+---+------+----------+----+------+----------+-----------+-----------------+----------------------------------+
|  1|  John|1992-05-12|  30| 70000|        IT|       true|          83000.0|                             false|
|  2| Alice|1997-02-28|  25| 90000|   Finance|       true|          72400.0|                              true|
|  2| Alice|1997-02-28|  25| 60000|        HR|       true|69333.33333333333|                             false|
|  3|   Bob|      null|null| 80000|        IT|       true|          83000.0|                             false|
|  4| Emily|1994-11-22|  28| 65000|   Finance|       true|          72400.0|                             false|
|  4| Emily|1994-11-22|  28| 70000|   Finance|       true|          72400.0|                            